In [1]:
# import os
# import sys

# module_path = os.path.abspath(os.path.join('../src'))
# print(module_path)
# if module_path not in sys.pathF:
#     sys.path.append(module_path)

/home/enis/projects/nna/src


In [2]:
from nna.fileUtils import list_files

from pathlib import Path
from nna.pre_process_func import read_queue

import pandas as pd

In [4]:
# originally from ../src/notes.md

* start with screen
* get GPU Memory so I can use it later
* one file file at a time (~2 gb)
  * I will divide files to 1 hour slices
  * I will use 25 cpus so 25 processes
  * embeddings file name will be filename_embeddings.



1) create a text file with paths to input audio files (one full path per line)
 
* `find /tank/data/nna/real/ -iname "*flac" > flacfiles.txt`
* or following code, list_files function


In [4]:
datafiles_path="/home/enis/projects/nna/data/"


In [293]:
search_path="/tank/data/nna/real/"
ignore_folders=["/tank/data/nna/real/ivvavik/","/tank/data/nna/real/stinchcomb/dups/","/tank/data/nna/real/stinchcomb/excerpts/"]

files_path_list=list_files(search_path,ignore_folders)
files_path_list=list(set(files_path_list))

datafiles_path="/home/enis/projects/nna/data/"
input_filename = "ExperimentRunV5.txt"

thepath=datafiles_path+input_filename
with open(thepath,"w") as myfile:
    myfile.writelines("\n".join(files_path_list))
    

2) update input, output directory, input list,  in /nna/src/params.py

3-1) sync files 

`rsync -av --recursive --update ./ enis@crescent:/home/enis/projects/nna/ `

3-2) run codes
```bash
python pre_process.py &>> job_logs/logs.txt; python slack_message.py -t "cpu job ended" -f job_logs/logs.txt &

python watch_VGGish.py &>> job_logs/logs.txt; python slack_message.py -t "gpu job stopped" & 
```
4) to re-run, update the code and remove temporary flac files

`rsync -av --recursive --update /Users/berk/Documents/workspace/speech_audio_understanding/src/ enis@crescent:/home/enis/projects/nna/`

`find /scratch/enis/data/nna/real/ -iname "*flac"  -delete`
`find /scratch/enis/data/nna/real/ -path "*/*_segments*/*" -delete`
`find /scratch/enis/data/nna/real/ -name "*_segments" -type d -delete`

also remove `job_logs/pre_processing_queue.csv` if jobs left unfinished

5) tracking progress 
```bash
cat job_logs/pre_processing_queue.csv | wc -l; cat job_logs/pre_processed_queue.csv | wc -l; cat job_logs/VGGISH_processing_queue.csv | wc -l; cat job_logs/vggish_embeddings_queue.csv | wc -l; 
du -hs /scratch/enis/data/
```
6) backup
```bash
tar cf - /scratch/enis/data/nna/backup/NUI_DATA/ -P | pv -s $(du -sb /scratch/enis/data/nna/backup/NUI_DATA/ | awk '{print $1}') | gzip > embeddings_backup.tar.gz
```


In [9]:
q1=read_queue("../src/job_logs/pre_processing_queue.csv")
q2=read_queue("../src/job_logs/pre_processed_queue.csv")
q3=read_queue("../src/job_logs/VGGISH_processing_queue.csv")
q4=read_queue("../src/job_logs/vggish_embeddings_queue.csv")
q5=read_queue("../src/job_logs/Audioset_processing_queue.csv")
q6=read_queue("../src/job_logs/Audioset_output_queue.csv")

CABLE=read_queue("../src/job_logs/_CABLE_output_queue.csv")



In [142]:
q1set=set(["/".join(i.split("/")[-1:])[:-5] for i in q1])
q2set=set(["/".join(i.split("/")[-2:-1])[:-13] for i in q2])


In [144]:
print(q1set.difference(q2set))
print(q2set.difference(q1set))

{'SINP-05_20190709_090000', 'S4A10252_20190529_171602', 'S4A10259_20190514_183000'}


In [5]:
# files without results

from visUtils import standardPathStyle
import pandas as pd

file_properties_df_ST=pd.read_pickle("../data/stinchcomb_dataV1.pkl")
file_properties_df=pd.read_pickle("../data/realdata_v2No_stinchcomb.pkl")
subDirectoryAddon="_vgg"
# subDirectoryAddon="_CABLE"
# subDirectoryAddon="_CABLE"
subDirectoryAddons=["_CABLE","_RUNNINGWATER","_INSECT", "_RAIN", "_WATERBIRD", "_WIND", "_SONGBIRD", "_AIRCRAFT"]
notDoneFilesDict={}
for subDirectoryAddon in subDirectoryAddons:
    print(subDirectoryAddon)
    notDoneFiles=[]
    
    for afile,row in file_properties_df.iterrows():
        checkFile=standardPathStyle("/scratch/enis/data/nna/real/",row,subDirectoryAddon=subDirectoryAddon,fileNameAddon="")
        if not checkFile.exists():
#             print(afile)
            notDoneFiles.append(afile)

    for afile,row in file_properties_df_ST.iterrows():
        checkFile=standardPathStyle("/scratch/enis/data/nna/real/",row,subDirectoryAddon=subDirectoryAddon,fileNameAddon="")
        if not checkFile.exists():
    #         print(afile)
#             print(checkFile)
            notDoneFiles.append(afile)
    notDoneFiles.sort()
    notDoneFiles=[str(i) for i in notDoneFiles]
    notDoneFilesDict[subDirectoryAddon]=notDoneFiles[:]

    

# input_filename = "ExperimentRunV6.txt"

# thepath=datafiles_path+input_filename
# with open(thepath,"w") as myfile:
#     myfile.writelines("\n".join(notDoneFiles))
    

### FIND missing prediction results

In [3]:
import glob
inputCSV=[]
for i in ["ivvavik","anwr",  "prudhoe" , "stinchcomb"]:
    x=glob.glob( f"/scratch/enis/data/nna/real/{i}/**/*_rawembeddings*.npy",recursive=True)
    print(i,len(x))
    inputCSV.extend(x)


save_to_csv("../src/job_logs/inferenceRun3.csv",[[str(afile)] for afile in inputCSV])


ivvavik 27852
anwr 8509
prudhoe 8228
stinchcomb 19376


In [ ]:
# _preprocessed.npy
search_path="/tank/data/nna/real/"
ignore_folders=["/tank/data/nna/real/ivvavik/","/tank/data/nna/real/stinchcomb/dups/","/tank/data/nna/real/stinchcomb/excerpts/"]

files_path_list=list_files(search_path,ignore_folders)
files_path_list=list(set(files_path_list))

datafiles_path="/home/enis/projects/nna/data/"
input_filename = "ExperimentRunV5.txt"

thepath=datafiles_path+input_filename
with open(thepath,"w") as myfile:
    myfile.writelines("\n".join(files_path_list))
    

### Run predictions

In [ ]:

inputCsvFile="./job_logs/inferenceRun3.csv"
outputLogs="job_logs/MLlogs.txt"
i=0
python3 watchMLInference.py --inputCsv $inputCsvFile --modelPath "./assets/sklearnModels/Cable_Gaussian Process_Raw_Concat_2020-03-02--14-06.joblib"  --modelName "_CABLE" &>> $outputLogs & 
pids[i]=$! && ((i=i+1));
python3 watchMLInference.py --inputCsv $inputCsvFile --modelPath "./assets/sklearnModels/Running Water_Neural Net_Raw_Concat_2020-03-02--14-06.joblib"  --modelName "_RUNNINGWATER" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
python3 watchMLInference.py --inputCsv $inputCsvFile --modelPath "./assets/sklearnModels/Insect_Linear SVM_Raw_Concat_2020-03-02--14-06.joblib"  --modelName "_INSECT" &>> $outputLogs & 
pids[i]=$! && ((i=i+1));
python3 watchMLInference.py --inputCsv $inputCsvFile --modelPath "./assets/sklearnModels/Rain_Gaussian Process_Raw_Concat_2020-03-02--14-06.joblib"  --modelName "_RAIN" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
python3 watchMLInference.py --inputCsv $inputCsvFile --modelPath "./assets/sklearnModels/Water Bird_RBF SVM_Raw_many2one_2020-03-02--14-06.joblib"  --modelName "_WATERBIRD" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
python3 watchMLInference.py --inputCsv $inputCsvFile --modelPath "./assets/sklearnModels/Wind_Neural Net_Raw_many2one_2020-03-02--14-06.joblib"  --modelName "_WIND" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
python3 watchMLInference.py --inputCsv $inputCsvFile --modelPath "./assets/sklearnModels/Songbird_Neural Net_Raw_many2one_2020-03-02--14-06.joblib"  --modelName "_SONGBIRD" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
python3 watchMLInference.py --inputCsv $inputCsvFile --modelPath "./assets/sklearnModels/Aircraft_AdaBoost_Raw_Average_2020-03-02--14-06.joblib"  --modelName "_AIRCRAFT" &>> $outputLogs &
pids[i]=$! && ((i=i+1));
for pid in ${pids[*]}; do
    wait $pid
done;
python3 slack_message.py -t "MLInference job ended" -f $outputLogs &


    

In [24]:
# set(inputCSV)
import time
logsPath="../src/job_logs/"
inputCSVset=set(inputCSV)
for subDirectoryAddon in subDirectoryAddons:
    csvFile=logsPath+subDirectoryAddon+"_output_queue.csv"
    print(subDirectoryAddon)
    csvFileSet=set(read_queue(csvFile))
    print(len(inputCSVset.difference(csvFileSet)))
#     for i in inputCSVset.difference(csvFileSet):
#         print(i)


_CABLE
0
_RUNNINGWATER
0
_INSECT
0
_RAIN
0
_WATERBIRD
0
_WIND
0
_SONGBIRD
0
_AIRCRAFT
0
